In [1]:
import os
import cv2
from skimage.feature import hog
from sklearn.svm import SVC

import tensorflow as tf
from tensorflow import keras

from keras.models import Sequential
from keras import Input
from keras.layers import Dense

import pandas as pd
import numpy as np

from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt

In [2]:
# import kagglehub

# # Download latest version
# path = kagglehub.dataset_download("rashikrahmanpritom/plant-disease-recognition-dataset")

# print("Path to dataset files:", path)

In [3]:
train_path = "./dataset/Train/Train"
test_path = "./dataset/Test/Test"

In [4]:
def load_images_from_folder(folder):
    images = []
    labels = []
    for label in os.listdir(folder):  
        class_folder = os.path.join(folder, label)
        if os.path.isdir(class_folder): 
            for filename in os.listdir(class_folder):
                img_path = os.path.join(class_folder, filename)
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Convert to grayscale
                # Skip unreadable images
                if img is not None:
                    
                    img = cv2.resize(img, (256, 256)) # Resize to 256x256
                    images.append(img)
                    labels.append(label) 
    return images, labels

In [5]:
train_images, train_labels = load_images_from_folder(train_path)
test_images, test_labels = load_images_from_folder(test_path)

In [6]:
def extract_hog_features(images):
    features = []
    for img in images:
       
        hog_feature = hog(
            img,
            orientations=12,                 
            pixels_per_cell=(16, 16),       
            cells_per_block=(2, 2),          
            block_norm='L2-Hys'              
        )
        features.append(hog_feature)
    return np.array(features)

In [7]:
X_train = extract_hog_features(train_images)
X_test = extract_hog_features(test_images)

In [8]:
y_train = np.array(train_labels)
y_test = np.array(test_labels)

In [9]:
svm_model = SVC(kernel='linear', random_state=42)
svm_model.fit(X_train, y_train)

SVC(kernel='linear', random_state=42)

In [10]:
y_pred = svm_model.predict(X_test)

In [11]:
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 58.00%

Classification Report:
              precision    recall  f1-score   support

     Healthy       0.56      0.70      0.62        50
     Powdery       0.73      0.64      0.68        50
        Rust       0.47      0.40      0.43        50

    accuracy                           0.58       150
   macro avg       0.58      0.58      0.58       150
weighted avg       0.58      0.58      0.58       150

